# Mancala Game

In [21]:
#not used... yet
from IPython.display import clear_output
#clear_output()

In [22]:
import random
import copy

In [23]:
#objeto Tablero
    # tiene un tablero y mancalas
    # metodo para realizar jugada de P1 o P2
    # metodo para saber si fila x == 0
    # metodo para imprimir en pantalla el estado del juego

class Tablero:
    def __init__(self):
        #inicializa el tablero y mancala
        self.mancala = [0,0]
        self.board = [0]*2
        for i in range(2):
            self.board[i] = [4]*6
        self.over = False
    
    
    #devuelve True si la fila seleccionada (i [0,1]) está vacía
    def fila_Vacia(self,i):
        for j in range(6):
            if(self.board[i][j] != 0):
                return False
        return True
    
    def gameOver(self):
        #print("NOT gameOver method")
        #print("filas: ", self.board)
        if(self.fila_Vacia(0) or self.fila_Vacia(1)):
            self.over = True
            #print("gameOver method")
    
    #realiza una jugada
    def makePlay(self, player, position):
        _position = position
        tokens = self.board[player][position] #obtengo todas las fichas en la casilla
        self.board[player][position] = 0 #remuevo todas las fichas de la casilla
        direction = player #0: backward ; 1: forward            
        while(tokens > 0):
            if(direction == 0):
                (tokens, _position) = self.run_backward(player,_position,tokens)
                direction = 1
            else:
                (tokens, _position) = self.run_forward(player,_position,tokens)
                direction = 0
        #print("last position: ",_position)
        self.gameOver() ## verificar si se ha acabado el juego
        return direction, _position #### position tupla [fila, position]
                
                
            
    def run_backward(self, player, position, tokens):
        fill_c = position - 1 #la primera posicion a llenar
        _tokens = tokens
        #se toma en cuenta -1 por ser la casilla mancala para jugador 0
        while((_tokens > 0) and (fill_c >-2)):
            #not mancala
            if(fill_c >-1):
                self.board[0][fill_c] += 1
                _tokens -= 1
            #mancala
            else:
                if(player == 0):
                    self.mancala[0] += 1
                    _tokens -= 1
            fill_c -= 1 #next position
        return (_tokens, fill_c+1)
    
    def run_forward(self, player, position, tokens):
        fill_c = position + 1
        _tokens = tokens
        #se toma en cuenta 6 por ser la casilla mancala para jugador 1
        while((_tokens > 0) and (fill_c < 7)):
            #not mancala
            if(fill_c < 6):
                self.board[1][fill_c] += 1
                _tokens -= 1
            #mancala
            else:
                if(player == 1):
                    self.mancala[1] += 1
                    _tokens -= 1
            fill_c += 1 #next position
        return (_tokens,fill_c-1)
    
    def showBoard(self):
        #score
        print("_______________________ *** _______________________")
        print("Jugador 1: ", self.mancala[0],"\t\t\tJugador 2: ",self.mancala[1],"\n\n")
        #board
        print("\t", end = '')
        for j in range(6):
            print(self.board[0][j], end = '')
            print("\t", end = '')
        print("\n", self.mancala[0], "\t\t\t\t\t\t\t", self.mancala[1])
        
        print("\t", end = '')
        for j in range(6):
            print(self.board[1][j], end = '')
            print("\t", end = '')
        print("\n")
                    
    

In [24]:
#Reglas
### Recibe tablero y next move (intento de next move)
### Retorna tablero y si se puede hacer el next move
def ValideMove(tablero, player, next_move):
    # evaluar si es posible realizar el next_move
    if(tablero.board[player][next_move] != 0):
        return True
    else:
        return False
    
def PossibleMoves(tablero, player):
    #print("player: ", player, "\ttablero: ", tablero.board)
    possible_moves = []
    for i in range(6):
        if(ValideMove(tablero, player, i)):
            possible_moves.append(i)
    return possible_moves
    
def Reglas(tablero, player, next_move):
    if(player == 0):
        adversario = 1
    else:
        adversario = 0
    
    if(tablero.over):
        if(tablero.fila_Vacia(0)):
            fila_NOvacia = 1
        else:
            fila_NOvacia = 0
            
        # la fila del adversario está vacía
        # se traslandan los tokens de la fila del player a su mancala
        tokens_1 = tablero.mancala[fila_NOvacia] #tokens en mancala del ganador
        tokens_2 = sum(tablero.board[fila_NOvacia])
        tablero.mancala[fila_NOvacia] = tokens_1 + tokens_2
        
        # se eliminan los tokens de la fila no vacia
        for i in range(6):
            tablero.board[fila_NOvacia][i] = 0
    
    [dire, last] = tablero.makePlay(player, next_move)
    if(last >= 0 and last <=5):
        if(tablero.board[player][last] == 1 and dire == adversario): #quiere decir que antes estaba vacío y está del lado del jugador (dire lógica inversa)
            #ROBAR FICHAS
            #print("holis")
            tokens_a = tablero.board[adversario][last]
            if(tokens_a > 0):
                tablero.board[adversario][last] = 0          #se roban fichas del adversario
                tablero.board[player][last] = 0              #ficha del jugador al mancala
                tablero.mancala[player] += tokens_a + 1
        
    if(last == -1 or last == 6): # último token en mancala del jugador
        next_player = player
        #print("----------->>   MANCALA!!!   <<-------------------")
    else:
        next_player = adversario
        
            
    return next_player # necesita retornar tablero? por ahora no

In [25]:
def Game_1(start):
    # determinar primer jugador solo si es el main
    ## pasa a ser código fuera del método
    if(start):
        player = random.randint(0,1)
        if(player == 0):
            print("La inteligencia artificial iniciará este juego")
        else:
            print("Tienes el primer turno")
    
    #tablero se genera fuera del método
    tablero = Tablero()    
    
    while(not tablero.over):
        #check if it's simulation
        clear_output()
        tablero.showBoard()
        #IA block
        if(player == 0):
            possible_moves = []
            for i in range(6):
                if(ValideMove(tablero, player, i)):
                    possible_moves.append(i)
            #eliminar para el otro método
            print("Movimientos posibles", possible_moves)
            next_move = int(input("Juego computadora "))
            #next_move = IA(tablero, possible_moves)
        #human
        else:
            valido = False
            while(not valido):
                x = int(input("¿Qué casilla deseas mover? "))
                if(ValideMove(tablero,player,x)):
                    next_move = x
                    valido = True
                else:
                    print("No hay fichas en esa casilla, elige otra.")
                    valido = False
        tablero.gameOver()
                    
        #tablero.makePlay(player,next_move)
        player = Reglas(tablero, player, next_move)
    
    print("GAME OVER")
            
            

#objeto IA
    # tiene metodo para realizar turno
    # tiene metodo para realizar monte carlo
    
    # por ahora tablero original es game
    # copia de game es tablero
    tablero0 = game
    for i in range(6):
        for j in range(n):
            ran = random.randint(0,5)
            tablero0.makePlay(i,ran)
            while(not game.over):
                u1 = random.randint(0,5)
                if(Game(tablero, u1)[0]):
                    tablero
                u2 = random.randint(0,5)
                Game(tablero)
    


In [27]:
def Game(tablero, init_player, simulation):
    tableroBase = copy.deepcopy(tablero) #copio el tablero
    player = init_player
    jugadas = []
    ganador = -1 #para que no este un jugador predestinado :P
    flag = True
    
    while(flag):
        #tableroBase = copy.deepcopy(tablero) #copio el tablero
        if(not simulation):
            #clear_output()
            tablero.showBoard()
            #Artificial intelligence block
            if(player == 0):
                possible_moves = PossibleMoves(tablero,player)
                print("possible_moves", possible_moves)
                next_move = IA(tablero, possible_moves, 1)
            #"human" block
            else:
                valido = False
                while(not valido and len(PossibleMoves(tablero,player))>0):
                    x = int(input("¿Qué casilla deseas mover? "))
                    if(ValideMove(tablero,player,x)):
                        next_move = x
                        valido = True
                    else:
                        print("No hay fichas en esa casilla, elige otra.")
                        valido = False
            
        else:
            print("TABLERO BASE")
            #tableroBase.showBoard()
            print("fin TABLERO BASE")
            possible_moves = PossibleMoves(tableroBase,player)
            #print("simulacion: ", possible_moves)
            if(len(possible_moves)>0):
                move = random.randint(0,len(possible_moves)-1)
                next_move = possible_moves[move]
            #print("selected: ", next_move)
        
        if(simulation):
            flag = not tableroBase.over
            player = Reglas(tableroBase, player, next_move)
            lista = tableroBase.mancala
            ganador = lista.index(max(tableroBase.mancala))
            #print("simulation")
            #tableroBase.showBoard()
        else:
            # tablero.showBoard()
            flag = not tablero.over
            print("NEXT_MOVE", next_move)
            player = Reglas(tablero, player, next_move)
            lista = tablero.mancala
            ganador = lista.index(max(tablero.mancala))
            
        tableroBase.gameOver()
        tablero.gameOver()
        jugadas.append((player, next_move))
    
    if(not simulation):
        print("El ganador fue: ", ganador)
        print("*******************************")
        print("*---- G A M E  *  O V E R ----*")
        print("*******************************")

    return ganador, jugadas
                

                


In [28]:
def IA(tablero_O, possible_moves, n):    
    tablero = copy.deepcopy(tablero_O)
    wins = [] #cada posición corresponde la posición de posibles movimientos
    for i in possible_moves:
        tablero.makePlay(0,i) #hago la jugada 
        #se simulan n juegos del tablero completo despues del movimiento
        cont_wins = 0
        for j in range(n):
            tablero_simulado = copy.deepcopy(tablero)
            #print("iteración: ", j)
            winner = Game(tablero_simulado, 1, True)[0]
            if(winner == 0):
                cont_wins += 1
        wins.append(cont_wins)
    
    # seleccionar jugada ganadora
    move = wins.index(max(wins))
    next_move = possible_moves[move]
    
    return next_move
                
                
            

#test block
game = Tablero()
game.showBoard()

game.makePlay(0,5)
game.showBoard()
game.makePlay(0,3)
game.showBoard()
game.makePlay(0,2)
game.showBoard()
game.makePlay(0,1)
game.showBoard()
game.makePlay(0,0)
game.showBoard()
game.makePlay(0,1)
game.showBoard()


In [ ]:
#Game_1(True)

In [29]:
# inicialización del juego
player = random.randint(0,1)
if(player == 0):
    print("La inteligencia artificial iniciará este juego")
else:
    print("Tienes el primer turno")
    
# TABLERO DE JUEGO
tablero = Tablero()

Game(tablero, player, False)

La inteligencia artificial iniciará este juego
_______________________ *** _______________________
Jugador 1:  0 			Jugador 2:  0 


	4	4	4	4	4	4	
 0 							 0
	4	4	4	4	4	4	

possible_moves [0, 1, 2, 3, 4, 5]
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BAS

¿Qué casilla deseas mover? 1
NEXT_MOVE 1
_______________________ *** _______________________
Jugador 1:  2 			Jugador 2:  7 


	1	0	0	4	5	5	
 2 							 7
	6	0	0	6	6	6	

possible_moves [0, 3, 4, 5]
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLER

¿Qué casilla deseas mover? 2
NEXT_MOVE 2
_______________________ *** _______________________
Jugador 1:  2 			Jugador 2:  16 


	2	1	2	0	8	0	
 2 							 16
	0	1	0	0	8	8	

possible_moves [0, 1, 2, 4]
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABLERO BASE
TABLERO BASE
fin TABL

UnboundLocalError: local variable 'next_move' referenced before assignment